In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
CLIENT_ID = 'CR0OYVFLX2GO0ABW5RDUEYOLDENDY0JUABRL1OBKXHEWSXHX' # your Foursquare ID
CLIENT_SECRET = 'DHUCYB5DFBS2TNUYYOUPUEV3A0FHEDMLOM0RLNZ0K3XJXU53' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version
radius = 5
LIMIT = 5

In [4]:
def drop_repeated_postcode(data):
    temp_list = []
    for i in range(1,data.shape[0]):
        if data.iloc[i,1] == data.iloc[i-1,1]:
            temp_list.append(i)
    return data.drop(temp_list).reset_index(drop = True)

def make_address(column, added_city):
    for i in range(column.shape[0]):
        column.iloc[i,0] = column.iloc[i,0] + ',' + added_city
    return column

def get_coordinate(address):
    address = address
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    return latitude, longitude

def add_columns_for_coordiate(data):
    data['Latitude'] = None
    data['Longitude'] = None
    
def get_listof_cooridate(data):
    latitude_list = []
    longitude_list = []
    for i in data['District']:
        latitude, longitude = get_coordinate(i)
        latitude_list.append(latitude)
        longitude_list.append(longitude)
        time.sleep(90)
    data['Latitude'] = latitude_list
    data['Longitude'] = longitude_list
    return data



In [5]:
#Collect data

#Hong Kong 
HK = pd.read_html('https://en.wikipedia.org/wiki/Districts_of_Hong_Kong')[5]

#Singapore
SG = pd.read_html('https://en.wikipedia.org/wiki/Postal_codes_in_Singapore')[0]

#Shanghai 
shanghai = pd.read_excel('Shanghai.xlsx')

#Beijing 
beijing = pd.read_excel('Beijing.xlsx')

#Taipei
TP = pd.read_html('https://en.wikipedia.org/wiki/Postal_codes_in_Taiwan')[1]

In [6]:
HK = HK[['District', 'Region']]
SG = SG[['Postal district', 'General location']]
shanghai = shanghai[['District/County in English', 'Postal Code']]
shanghai = drop_repeated_postcode(shanghai)
beijing = beijing[['District/County in English', 'Postal Code']]
beijing = drop_repeated_postcode(beijing)
TP = TP.iloc[:,[0,1]]

# Make full address -- District + City name 

# Working on it

In [7]:
HK[['District']] = make_address(HK[['District']], 'Hong Kong')

for i in range(SG.shape[0]):
    SG.iloc[i,1] = SG.iloc[i,1].split(',')[0]  
SG[['General location']] = make_address(SG[['General location']], 'Singapore')

shanghai[['District/County in English']] = make_address(shanghai[['District/County in English']], 'Shanghai')

beijing[['District/County in English']] = make_address(beijing[['District/County in English']], 'Beijing')

TP[['Division name']] =  make_address(TP[['Division name']], 'Taipei')

/Users/cloudli/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/cloudli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [8]:
SG = SG[['General location', 'Postal district']]
SG.columns = ['District', 'Postal District']
shanghai.columns = ['District', 'Postal Code']
beijing.columns = ['District', 'Postal Code']
TP = TP.iloc[:,[1,0]]
TP.columns = ['District', 'Postal Code']

# 1. Get coordiate for each dataframe (total 5)
# 2. Put it in two new columns

In [10]:
add_columns_for_coordiate(HK)
add_columns_for_coordiate(SG)
add_columns_for_coordiate(shanghai)
add_columns_for_coordiate(beijing)
add_columns_for_coordiate(TP)

In [12]:
HK

,District,Region,Latitude,Longitude
0,"Central and Western,Hong Kong",Hong Kong Island,None,None
1,"Eastern,Hong Kong",Hong Kong Island,None,None
2,"Southern,Hong Kong",Hong Kong Island,None,None
3,"Wan Chai,Hong Kong",Hong Kong Island,None,None
4,"Sham Shui Po,Hong Kong",Kowloon,None,None
5,"Kowloon City,Hong Kong",Kowloon,None,None
6,"Kwun Tong,Hong Kong",Kowloon,None,None
7,"Wong Tai Sin,Hong Kong",Kowloon,None,None
8,"Yau Tsim Mong,Hong Kong",Kowloon,None,None
9,"Islands,Hong Kong",New Territories,None,None


In [13]:
HK = get_listof_cooridate(HK)
time.sleep(300)
SG = get_listof_cooridate(SG)
time.sleep(300)
shanghai = get_listof_cooridate(shanghai)
time.sleep(300)
beijing = get_listof_cooridate(beijing)
time.sleep(500)
TP = get_listof_cooridate(TP)

/Users/cloudli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Central and Western,Hong Kong are 22.2850394, 114.1583819.
The geograpical coordinate of Eastern,Hong Kong are 43.08984985, 131.96775944733844.
The geograpical coordinate of Southern,Hong Kong are 60.1856153, 24.9223967.
The geograpical coordinate of Wan Chai,Hong Kong are 22.2790146, 114.1724833.
The geograpical coordinate of Sham Shui Po,Hong Kong are 22.32819, 114.1608545.
The geograpical coordinate of Kowloon City,Hong Kong are 22.3301595, 114.1899368.
The geograpical coordinate of Kwun Tong,Hong Kong are 22.312937, 114.22561.
The geograpical coordinate of Wong Tai Sin,Hong Kong are 22.34165365, 114.19385869532955.
The geograpical coordinate of Yau Tsim Mong,Hong Kong are 22.3074036, 114.16552591155013.
The geograpical coordinate of Islands,Hong Kong are 39.8996249, 4.0778475.
The geograpical coordinate of Kwai Tsing,Hong Kong are 22.34100685, 114.10428549895929.
The geograpical coordinate of North,Hong Kong are 51.1990369, 6.8409999.
The geograpical c

GeocoderTimedOut: Service timed out

In [19]:
TP = get_listof_cooridate(TP)

/Users/cloudli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Zhongzheng District,Taipei are 25.0323611, 121.518267.
The geograpical coordinate of Datong District,Taipei are 25.065986, 121.515514.
The geograpical coordinate of Zhongshan District,Taipei are 25.0643611, 121.533468.
The geograpical coordinate of Songshan District,Taipei are 25.0498852, 121.5772719.
The geograpical coordinate of Daan District,Taipei are 25.0265152, 121.534395.
The geograpical coordinate of Wanhua District,Taipei are 25.0319327, 121.4993322.
The geograpical coordinate of Xinyi District,Taipei are 25.0375198, 121.5636796.
The geograpical coordinate of Shilin District,Taipei are 25.0918396, 121.5242068.
The geograpical coordinate of Beitou District,Taipei are 25.1319307, 121.4985934.
The geograpical coordinate of Neihu District,Taipei are 25.069664, 121.5889983.
The geograpical coordinate of Nangang District,Taipei are 25.0545779, 121.6065999.
The geograpical coordinate of Wenshan District,Taipei are 24.989786, 121.570458.


In [25]:
HK = HK.drop([1,2,9,11])

In [27]:
HK

,District,Region,Latitude,Longitude
0,"Central and Western,Hong Kong",Hong Kong Island,22.285039,114.158382
3,"Wan Chai,Hong Kong",Hong Kong Island,22.279015,114.172483
4,"Sham Shui Po,Hong Kong",Kowloon,22.328190,114.160854
5,"Kowloon City,Hong Kong",Kowloon,22.330160,114.189937
6,"Kwun Tong,Hong Kong",Kowloon,22.312937,114.225610
7,"Wong Tai Sin,Hong Kong",Kowloon,22.341654,114.193859
8,"Yau Tsim Mong,Hong Kong",Kowloon,22.307404,114.165526
10,"Kwai Tsing,Hong Kong",New Territories,22.341007,114.104285
12,"Sai Kung,Hong Kong",New Territories,22.382249,114.272828
13,"Sha Tin,Hong Kong",New Territories,22.381056,114.188879


In [38]:
TP.to_csv('TP.csv', index = None)

In [39]:
pd.read_csv('TP.csv')

,District,Postal Code,Latitude,Longitude
0,"Zhongzheng District,Taipei",100,25.032361,121.518267
1,"Datong District,Taipei",103,25.065986,121.515514
2,"Zhongshan District,Taipei",104,25.064361,121.533468
3,"Songshan District,Taipei",105,25.049885,121.577272
4,"Daan District,Taipei",106,25.026515,121.534395
5,"Wanhua District,Taipei",108,25.031933,121.499332
6,"Xinyi District,Taipei",110,25.037520,121.563680
7,"Shilin District,Taipei",111,25.091840,121.524207
8,"Beitou District,Taipei",112,25.131931,121.498593
9,"Neihu District,Taipei",114,25.069664,121.588998


In [56]:
HK_Venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Central and Western,Hong Kong",22.285039,114.158382,Apple ifc mall,22.284602,114.159072,Electronics Store
1,"Central and Western,Hong Kong",22.285039,114.158382,Four Seasons Hotel Hong Kong (香港四季酒店),22.286554,114.156929,Hotel
2,"Central and Western,Hong Kong",22.285039,114.158382,ifc mall (國際金融中心商場),22.284613,114.158156,Shopping Mall
3,"Central and Western,Hong Kong",22.285039,114.158382,Pool Terrace (池畔餐廳),22.286218,114.157583,Snack Place
4,"Central and Western,Hong Kong",22.285039,114.158382,The Spa at Four Seasons,22.286279,114.157623,Spa
...,...,...,...,...,...,...,...
64,"Yuen Long,Hong Kong",22.442646,114.030434,Aya (彩),22.441018,114.030366,Ramen Restaurant
65,"Yuen Long,Hong Kong",22.442646,114.030434,亞玉豆腐花,22.444035,114.031049,Dessert Shop
66,"Yuen Long,Hong Kong",22.442646,114.030434,Ichininmae (一人前),22.442218,114.031530,Noodle House
67,"Yuen Long,Hong Kong",22.442646,114.030434,Man Fong Café (萬芳冰室),22.440765,114.031362,Hong Kong Restaurant
